In [1]:
import pickle as pkl

import polars as pl

from backtest_lib.market.polars_impl import PolarsPastView

close_df = pkl.load(open("./sp500_close.pkl", "rb"))
close_df = close_df.dropna(axis=1, how="any")
dates = close_df.index.values

close_pl = pl.from_pandas(close_df)
securities = close_pl.columns
close_prices_df = close_pl.with_columns(pl.Series("date", dates))
past_cost_prices = PolarsPastView.from_dataframe(close_prices_df)

past_cost_prices

PolarsPastView(by_security=PolarsBySecurity(_security_column_df=shape: (909, 497)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ AMD       ┆ AMZN      ┆ ALGN      ┆ ABBV      ┆ … ┆ VZ        ┆ ZBRA      ┆ ZBH       ┆ ZTS      │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ f64       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 150.24000 ┆ 170.40449 ┆ 648.04998 ┆ 135.41999 ┆ … ┆ 52.439999 ┆ 583.90002 ┆ 125.36893 ┆ 234.0299 │
│ 5         ┆ 5         ┆ 8         ┆ 8         ┆   ┆           ┆ 4         ┆ 5         ┆ 99       │
│ 144.41999 ┆ 167.52200 ┆ 623.5     ┆ 135.16000 ┆ … ┆ 53.470001 ┆ 587.59997 ┆ 126.64077 ┆ 225.1199 │
│ 8         ┆ 3         ┆           ┆ 4         ┆   ┆           ┆ 6         ┆ 8         ┆ 95       │
│ 136.149

In [2]:
from backtest_lib.market import MarketView, PastUniversePrices
from backtest_lib.market.polars_impl import PolarsPastView

df = past_cost_prices.by_security.as_df(show_periods=False)
cols = df.columns
ewma = (
    df.with_columns(pl.col(col).ewm_mean(com=2).alias(f"{col}_EWM") for col in cols)
    .with_columns((pl.col(col) / pl.col(f"{col}_EWM")) for col in cols)
    .select(cols)
    .with_columns(date=dates)
)

ewma_past_view = PolarsPastView.from_dataframe(ewma)

market_view = MarketView(
    prices=PastUniversePrices(close=past_cost_prices),
    signals={"ewma_ratio": ewma_past_view},
    periods=dates,
)

market_view.signals["ewma_ratio"].by_period[-1]

SeriesUniverseMapping(names=('AMD', 'AMZN', 'ALGN', 'ABBV', 'ALLE', 'AWK', 'ANET', 'AMT', 'APTV', 'GOOGL', 'ADBE', 'AMAT', 'APO', 'A', 'APA', 'AON', 'T', 'APD', 'ADM', 'AFL', 'AJG', 'AEP', 'AMGN', 'AXP', 'MO', 'ABT', 'LNT', 'ALL', 'ATO', 'AAPL', 'ARE', 'MMM', 'AME', 'AIZ', 'AMP', 'AES', 'ADSK', 'AKAM', 'ACGL', 'ABNB', 'AMCR', 'ADI', 'GOOG', 'ADP', 'APH', 'AEE', 'ACN', 'AOS', 'AIG', 'ALB', 'KMX', 'BIIB', 'CZR', 'BLDR', 'CBOE', 'BSX', 'CBRE', 'XYZ', 'AVGO', 'CDW', 'AZO', 'BXP', 'BMY', 'BK', 'BX', 'BG', 'COR', 'BR', 'BAC', 'CARR', 'TECH', 'BA', 'CAH', 'BLK', 'BAX', 'CPB', 'CVX', 'BF-B', 'CNP', 'BBY', 'BALL', 'CF', 'CAT', 'CPT', 'BKNG', 'CDNS', 'BRK-B', 'AVY', 'AXON', 'CNC', 'COF', 'CCL', 'CHRW', 'CHTR', 'SCHW', 'CRL', 'AVB', 'BKR', 'BRO', 'BDX', 'DVA', 'CPAY', 'DXCM', 'DDOG', 'CTSH', 'DAY', 'CSGP', 'CMG', 'CTVA', 'FANG', 'C', 'DG', 'COO', 'CINF', 'DELL', 'CMI', 'DECK', 'CTAS', 'CB', 'CCI', 'CTRA', 'CMCSA', 'CRWD', 'COP', 'CFG', 'ED', 'CVS', 'CAG', 'KO', 'GLW', 'DE', 'DLR', 'DHR', 'CSCO', 

In [3]:
from backtest_lib.portfolio import WeightedPortfolio
from backtest_lib.strategy import Decision


def ewma_strategy(universe, current_portfolio, market, ctx):
    latest_ewma_ratio = (
        market.signals["ewma_ratio"].by_security[list(universe)].by_period[-1]
    )

    avg_ewma_ratio = latest_ewma_ratio.mean()
    norm_ewma_ratio = latest_ewma_ratio / avg_ewma_ratio
    weights = norm_ewma_ratio / len(norm_ewma_ratio)
    return Decision(target=WeightedPortfolio(cash=0, holdings=weights))


decision = ewma_strategy(["AAPL", "MSFT"], None, market_view, None)
decision

Decision(target=WeightedPortfolio(cash=0, holdings=SeriesUniverseMapping(names=('AAPL', 'MSFT'), _data=shape: (2,)
Series: '2025-08-18 00:00:00.000000000' [f64]
[
	0.50242
	0.49758
], _scalar_type=<class 'float'>)), notes=None)

In [4]:
import polars as pl

from backtest_lib.backtest import Backtest
from backtest_lib.market.polars_impl import SeriesUniverseMapping

universe = tuple(past_cost_prices.by_security.as_df(show_periods=False).columns)
pf = WeightedPortfolio(
    cash=0,
    holdings=SeriesUniverseMapping.from_names_and_data(
        universe, pl.Series([1 / len(universe)] * len(universe))
    ),
)
backtest = Backtest(
    universe=universe,
    strategy=ewma_strategy,
    market_view=market_view,
    initial_portfolio=pf,
)

results = backtest.run()
print(f"total return: {(results.total_return - 1) * 100:.2f}%")

total return: -45.10%


In [5]:
def index_strategy(universe, current_portfolio, market, ctx) -> Decision:
    return Decision(pf)


benchmark_backtest = Backtest(
    universe=universe,
    strategy=index_strategy,
    market_view=market_view,
    initial_portfolio=pf,
)

results = benchmark_backtest.run()
print(f"total return: {(results.total_return - 1) * 100:.2f}%")

total return: -71.94%


In [6]:
import polars as pl

first_prices = past_cost_prices.by_period[0]
last_prices = past_cost_prices.by_period[-1]

avg_first_change = (
    past_cost_prices.by_period[1] / past_cost_prices.by_period[0]
).mean()

print("avg_first_change:", avg_first_change)

(last_prices / first_prices).mean()

avg_first_change: 1.0066474326975505


1.300570476326577

In [7]:
aapl_prices = past_cost_prices.by_security["AAPL"].as_series()

print(aapl_prices.pct_change() + 1)
print((aapl_prices.pct_change() + 1).cum_prod())

shape: (909,)
Series: 'AAPL' [f64]
[
	null
	0.987308
	0.9734
	0.983307
	1.000988
	…
	1.010872
	1.016024
	0.997643
	0.994888
	0.996977
]
shape: (909,)
Series: 'AAPL' [f64]
[
	null
	0.987308
	0.961046
	0.945003
	0.945937
	…
	1.261744
	1.281963
	1.278941
	1.272403
	1.268557
]


In [8]:
past_cost_prices.by_period[-1].mean()

225.9554230807053

In [9]:
df = past_cost_prices.by_security.as_df(show_periods=False)
cols = df.columns
ewma_weights = (
    df.with_columns(pl.col(col).ewm_mean(com=2).alias(f"{col}_EWM") for col in cols)
    .with_columns((pl.col(col) / pl.col(f"{col}_EWM").alias(col)) for col in cols)
    .select(cols)
    .with_columns(pl.mean_horizontal(cols).alias("avg_ratio"))
    .with_columns(((pl.col(col) / pl.col("avg_ratio")) / len(cols)) for col in cols)
    .select(cols)
    .with_columns(date=dates)
)

ewma_weights_past_view = PolarsPastView.from_dataframe(ewma_weights)
market_view = MarketView(
    periods=dates,
    prices=PastUniversePrices(close=past_cost_prices),
    signals={"precomputed_weights": ewma_weights_past_view},
)


def fast_ewma_strategy(universe, current_portfolio, market, ctx) -> Decision:
    return Decision(
        WeightedPortfolio(
            cash=0, holdings=market_view.signals["precomputed_weights"].by_period[-1]
        )
    )


fast_ewma_backtest = Backtest(
    universe=universe,
    strategy=fast_ewma_strategy,
    market_view=market_view,
    initial_portfolio=pf,
)

In [10]:
fast_ewma_backtest.run()

BacktestResults(initial_capital=1.0, portfolio_returns=[0.0, 0.006606689668792624, -0.017701573295444105, 0.002283281014003594, -0.0028716447460513054, -0.003900633929594118, 0.009516783846116959, 0.0013004390158793122, -0.006973685203875459, -0.001876582508926943, -0.017362420584538442, -0.009979558807293409, -0.011966393467873135, -0.015529102775208197, 0.007412823645312582, -0.011800852822109109, -0.007315950065024662, -0.006301508760997783, 0.01942394709401167, 0.018578659443446912, 0.007656985378622119, 0.006674514405162013, -0.016074146289719836, -0.00019590778930542955, 0.0007294243917379912, 0.008558064963797764, 0.017921792520800527, -0.01640836402784496, -0.013997574988262092, -0.007592851065367719, 0.016058360667234543, 0.002149943653067106, -0.01892861331891261, -0.004986426706272026, -0.0102226217199943, -0.01667099976168112, 0.01339608961379839, 0.026088685851553987, -0.004601129696778694, -0.016779821448505684, 0.022509502659300344, -0.004002404844043007, -0.007217864930

In [11]:
(
    df.with_columns(pl.col(col).ewm_mean(com=2).alias(f"{col}_EWM") for col in cols)
    .with_columns((pl.col(col) / pl.col(f"{col}_EWM")) for col in cols)
    .select(cols)
    .with_columns(pl.mean_horizontal(cols).alias("avg_ratio"))
    .with_columns(((pl.col(col) / pl.col("avg_ratio")) / len(cols)) for col in cols)
    .select(cols)
    .with_columns(date=dates)
)

AMD,AMZN,ALGN,ABBV,ALLE,AWK,ANET,AMT,APTV,GOOGL,ADBE,AMAT,APO,A,APA,AON,T,APD,ADM,AFL,AJG,AEP,AMGN,AXP,MO,ABT,LNT,ALL,ATO,AAPL,ARE,MMM,AME,AIZ,AMP,AES,ADSK,…,VRSN,TSN,WAB,WMB,WMT,TRV,VLO,GWW,TFC,USB,WBA,WRB,WAT,WFC,WST,ULTA,DIS,VMC,WBD,YUM,V,UAL,VTR,VST,UDR,WELL,VTRS,UPS,WEC,UNP,XEL,UHS,VZ,ZBRA,ZBH,ZTS,date
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,datetime[ns]
0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,…,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,0.002012,2022-01-03 00:00:00
0.001975,0.001993,0.001976,0.002005,0.002026,0.001986,0.001986,0.001997,0.002036,0.002004,0.001992,0.001999,0.002029,0.001979,0.002046,0.00201,0.002014,0.002017,0.002022,0.002033,0.00201,0.002007,0.002011,0.002032,0.002025,0.001988,0.002005,0.002026,0.002016,0.001997,0.002011,0.002018,0.00202,0.002015,0.002042,0.002009,0.001991,…,0.002002,0.002023,0.002052,0.002014,0.001992,0.002024,0.002021,0.002023,0.002032,0.002028,0.002013,0.002019,0.001989,0.002038,0.001976,0.002018,0.002002,0.002017,0.002051,0.002011,0.002011,0.00202,0.002028,0.001999,0.002019,0.002031,0.002016,0.002021,0.002005,0.002021,0.002012,0.002018,0.002023,0.002012,0.002015,0.001976,2022-01-04 00:00:00
0.001949,0.002001,0.001933,0.002033,0.002013,0.00201,0.001974,0.001952,0.002019,0.001977,0.001942,0.001993,0.001981,0.001995,0.00201,0.002026,0.002056,0.002051,0.002028,0.002041,0.00201,0.002039,0.002018,0.002031,0.002029,0.002014,0.002037,0.002038,0.002037,0.001994,0.00196,0.00203,0.002016,0.002023,0.002037,0.00198,0.001966,…,0.001996,0.002039,0.00204,0.002035,0.002035,0.002043,0.002032,0.002012,0.00203,0.002034,0.002042,0.002023,0.002002,0.002036,0.001948,0.001989,0.002022,0.002024,0.00204,0.002017,0.002019,0.002025,0.002031,0.002005,0.002007,0.002027,0.002039,0.002023,0.00204,0.002039,0.002042,0.002034,0.002048,0.001977,0.002029,0.001971,2022-01-05 00:00:00
0.001973,0.001995,0.00194,0.002016,0.001999,0.00199,0.001983,0.001976,0.002054,0.001989,0.001967,0.002018,0.001991,0.002004,0.002058,0.002007,0.00203,0.002008,0.002029,0.002048,0.002021,0.002021,0.002013,0.002028,0.002033,0.00201,0.002019,0.002038,0.002015,0.001979,0.001997,0.00201,0.002026,0.002012,0.00204,0.001986,0.001981,…,0.001992,0.00204,0.002024,0.002051,0.00202,0.002046,0.00205,0.002014,0.002064,0.002054,0.001993,0.002036,0.00201,0.002053,0.001982,0.001993,0.002028,0.002025,0.001993,0.002025,0.002012,0.002017,0.00203,0.002017,0.002007,0.002027,0.00203,0.002027,0.002018,0.002029,0.002018,0.001997,0.002025,0.001982,0.00201,0.00199,2022-01-06 00:00:00
0.001953,0.002,0.001925,0.002015,0.001994,0.001989,0.001968,0.002001,0.002008,0.001995,0.00198,0.001975,0.001992,0.001977,0.002046,0.00198,0.002036,0.002011,0.002037,0.002058,0.002016,0.002044,0.002028,0.002037,0.002043,0.002019,0.00202,0.002068,0.002012,0.001997,0.002001,0.002028,0.002001,0.002039,0.002034,0.001992,0.001988,…,0.001982,0.002035,0.002006,0.002065,0.002032,0.002067,0.002053,0.002006,0.002067,0.002061,0.002037,0.002055,0.001995,0.002068,0.001944,0.001994,0.002033,0.002012,0.002201,0.002009,0.002,0.00206,0.002035,0.00204,0.001991,0.002028,0.002047,0.002034,0.002031,0.002032,0.00203,0.002016,0.002035,0.001942,0.002007,0.001966,2022-01-07 00:00:00
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…